In [1]:
import os.path
import numpy as np
import pandas as pd

import bcolz
import dask.dataframe as dd

from time import time
from pandas.computation import expressions as expr

In [114]:
# set configure
path = "./DATA_FULL/"
filename = "output_MDS_ACTIVE_MLY_2016-09-01"
out_filename = "mds_mly_minus20160901"
relative_filename = path + filename + ".csv"

In [31]:
wants_cols = ['DATA_MONTH', 'MINING_DW_SUBSCR_NO', 'AGE', "MO_TTL_DURATION","MO_ONNET_DUR","MO_OFFNET_DUR","PSTN_DUR","MO_TTL_COUNT","MO_ONNET_CNT","MO_OFFNET_CNT","PSTN_CNT","MT_TTL_DURATION","MT_ONNET_DUR","MT_OFFNET_DUR","MT_PSTN_DUR","MT_TTL_COUNT","MT_ONNET_CNT","MT_OFFNET_CNT","MT_PSTN_CNT","MOC_FET_DUR","MOC_TWM_DUR","MOC_CHM_DUR","MOC_OTH_DUR","MOC_PSTN_DUR","MOST_MO_DUR","MTC_FET_DUR","MTC_TWM_DUR","MTC_CHM_DUR","MTC_PSTN_DUR","MOST_MT_DUR"]

In [20]:
# wants_cols = ['DATA_MONTH', 'MINING_DW_SUBSCR_NO', 'AGE', "MOST_MO_OPERATOR", "MOST_MT_OPERATOR"]

In [2]:
def bcolz_setting():
	# Global settings for bcolz and pandas (for experimenting, modify some of the lines below)
	bcolz.defaults.cparams['cname'] = 'blosclz'
	bcolz.defaults.cparams['clevel'] = 1
	bcolz.defaults.eval_vm = "numexpr"
	bcolz.blosc_set_nthreads(4)
	bcolz.numexpr.set_num_threads(4)
	expr.set_use_numexpr(True)
#	expr.set_numexpr_threads(1)

In [3]:
	bcolz.print_versions()

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
bcolz version:     1.1.2
NumPy version:     1.12.1
Blosc version:     1.11.2 ($Date:: 2017-01-27 #$)
Blosc compressors: ['blosclz', 'lz4', 'lz4hc', 'snappy', 'zlib', 'zstd']
Numexpr version:   2.6.2
Dask version:      0.14.3
Python version:    3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
Byte-ordering:     little
Detected cores:    8
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


In [4]:
	bcolz_setting()

C:\Users\VIPLAB\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: pandas.computation.expressions.set_use_numexpr is deprecated and will be removed in a future version.
you can toggle usage of numexpr via pandas.get_option('compute.use_numexpr')
  


In [32]:
# read csv file and print cost time
t0 = time()
df = pd.read_csv(relative_filename, usecols = wants_cols, error_bad_lines=False)
print("time for read csv: %.2f" % (time()-t0))


C:\Users\VIPLAB\AppData\Local\conda\conda\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (134,135,136,137,138,139,140,141,142,143) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


time for read csv: 28.97


In [81]:
# remove row which the value is null in column age
df = df.dropna(axis=0, subset =['AGE'])

In [134]:
# let nan == 0
df = df.fillna(0)
# "MO_ONNET_DUR" will have "Y/N" , replace it to 0
df[df["MO_ONNET_DUR"] == "Y"] = 0
df[df["MO_ONNET_DUR"] == "N"] = 0

In [135]:
# write to csv and no index
t0 = time()
df.to_csv(out_filename + ".csv", index=False)
print("time for ctable conversion: %.2f" % (time()-t0))

time for ctable conversion: 22.25


In [141]:
normalize_cols = df.columns[4:5]
for col in normalize_cols:
    print(col)
    df[col] = df[col].apply(np.float64)
    t0 = time()
#     print("MO_ONNET_DUR.mean() =", df[col].mean())
    print(df[col].describe())
    print("time for column sum: %.2f" % (time()-t0))


MO_ONNET_DUR
count    1.878076e+06
mean     2.644061e+03
std      1.669910e+04
min     -2.400000e+02
25%      0.000000e+00
50%      2.860000e+02
75%      1.627000e+03
max      4.323402e+06
Name: MO_ONNET_DUR, dtype: float64
time for column sum: 0.08


In [85]:
t0 = time()
#voice_pd = bcolz.ctable.fromdataframe(df, rootdir='mds_mly_dataframe.bcolz')
voice_pd = bcolz.ctable.fromdataframe(df)
print("time for ctable conversion: %.2f" % (time()-t0))

time for ctable conversion: 259.81


In [66]:
df

,DATA_MONTH,MINING_DW_SUBSCR_NO,AGE,MO_TTL_DURATION,MO_ONNET_DUR,MO_OFFNET_DUR,PSTN_DUR,MO_TTL_COUNT,MO_ONNET_CNT,MO_OFFNET_CNT,...,MOC_TWM_DUR,MOC_CHM_DUR,MOC_OTH_DUR,MOC_PSTN_DUR,MOST_MO_DUR,MTC_FET_DUR,MTC_TWM_DUR,MTC_CHM_DUR,MTC_PSTN_DUR,MOST_MT_DUR
0,2016-09-01,3133025292,59.0,327,244,53,30,8,5,1,...,0,53,0,30,244,158,0,131,0,158
1,2016-09-01,1110237261,32.0,2270,2028,125,117,25,16,6,...,82,43,0,117,2028,4735,239,79,166,4735
2,2016-09-01,3124977017,27.0,1027,0,260,0,23,0,7,...,0,260,0,0,260,74,50,253,0,253
3,2016-09-01,3134132753,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2016-09-01,3129664663,37.0,5839,5058,618,96,39,31,4,...,54,564,0,96,5058,2864,557,614,585,2864
5,2016-09-01,3113232074,38.0,15644,5483,7210,2384,234,130,68,...,903,3341,2966,2384,5483,4409,1388,2554,1326,4409
6,2016-09-01,3123789335,20.0,17612,15034,1961,209,120,91,10,...,929,18,1014,209,15034,41202,177,256,122,41202
7,2016-09-01,3126537000,38.0,3986,1510,1460,863,57,21,21,...,149,1311,0,863,1510,1146,755,1357,1467,1467
8,2016-09-01,3130233668,38.0,7158,1179,259,1665,55,17,10,...,234,10,15,1665,1665,299,488,419,456,488
9,2016-09-01,3130649928,NaN,7167,31,4438,2698,74,2,50,...,338,222,3878,2698,3878,9717,45,188,1378,9717


In [36]:
voice_pd

ctable((2000000,), [('DATA_MONTH', '<U10'), ('MINING_DW_SUBSCR_NO', '<i8'), ('AGE', '<f8'), ('MO_TTL_DURATION', 'O'), ('MO_ONNET_DUR', 'O'), ('MO_OFFNET_DUR', 'O'), ('PSTN_DUR', 'O'), ('MO_TTL_COUNT', 'O'), ('MO_ONNET_CNT', 'O'), ('MO_OFFNET_CNT', 'O'), ('PSTN_CNT', 'O'), ('MT_TTL_DURATION', 'O'), ('MT_ONNET_DUR', 'O'), ('MT_OFFNET_DUR', '<f8'), ('MT_PSTN_DUR', '<i8'), ('MT_TTL_COUNT', '<i8'), ('MT_ONNET_CNT', '<i8'), ('MT_OFFNET_CNT', '<i8'), ('MT_PSTN_CNT', '<i8'), ('MOC_FET_DUR', '<i8'), ('MOC_TWM_DUR', '<i8'), ('MOC_CHM_DUR', '<i8'), ('MOC_OTH_DUR', '<i8'), ('MOC_PSTN_DUR', '<i8'), ('MOST_MO_DUR', '<i8'), ('MTC_FET_DUR', '<i8'), ('MTC_TWM_DUR', '<i8'), ('MTC_CHM_DUR', '<i8'), ('MTC_PSTN_DUR', '<i8'), ('MOST_MT_DUR', '<i8')])
  nbytes: 676.84 MB; cbytes: 704.18 MB; ratio: 0.96
  cparams := cparams(clevel=1, shuffle=1, cname='blosclz', quantize=0)
[ ('2016-09-01', 3133025292,  59., '327', '244', '53', 30.0, 8, 5, 1, 2, 289, 158,  131., 0, 4, 3, 1, 0, 244, 0, 53, 0, 30, 244, 158, 0, 1